# [AI训练营]PaddleSeg实现语义分割Baseline

手把手教你基于PaddleSeg实现语义分割任务

------


# 一、作业任务

> 本次任务将基于PaddleSeg展开语义分割任务的学习与实践，baseline会提供PaddleSeg套件的基本使用，相关细节如有遗漏，可参考[10分钟上手PaddleSeg
](https://aistudio.baidu.com/aistudio/projectdetail/1672610?channelType=0&channel=0)

1. 选择提供的**五个数据集**中的一个数据集作为本次作业的训练数据，并**根据baseline跑通项目**

2. **可视化1-3张**预测图片与预测结果（本次提供的数据集没有验证与测试集，可将训练数据直接进行预测，展示训练数据的预测结果即可）


**加分项:**

3. 尝试**划分验证集**，再进行训练

4. 选择**其他网络**，调整训练参数进行更好的尝试


**PS**:PaddleSeg相关参考项目:

- [常规赛：PALM病理性近视病灶检测与分割基线方案](https://aistudio.baidu.com/aistudio/projectdetail/1941312)

- [PaddleSeg 2.0动态图：车道线图像分割任务简介](https://aistudio.baidu.com/aistudio/projectdetail/1752986?channelType=0&channel=0)

------

------

# 二、数据集说明

------

本项目使用的数据集是:[AI训练营]语义分割数据集合集，包含马分割，眼底血管分割，车道线分割，场景分割以及人像分割。

该数据集已加载到本环境中，位于:

**data/data103787/segDataset.zip**


In [ ]:
# unzip: 解压指令
# -o: 表示解压后进行输出
# -q: 表示静默模式，即不输出解压过程的日志
# -d: 解压到指定目录下，如果该文件夹不存在，会自动创建
!unzip -oq data/data103787/segDataset.zip -d segDataset

解压完成后，会在左侧文件目录多出一个**segDataset**的文件夹，该文件夹下有**5**个子文件夹：

- **horse -- 马分割数据**<二分类任务>

![](https://ai-studio-static-online.cdn.bcebos.com/2b12a7fab9ee409587a2aec332a70ba2bce0fcc4a10345a4aa38941db65e8d02)


In [ ]:
# tree: 查看文件夹树状结构
# -L: 表示遍历深度
!tree segDataset -L 2

segDataset
├── cocome
│   ├── Annotations
│   └── Images
├── facade
│   ├── Annotations
│   └── Images
├── FundusVessels
│   ├── Annotations
│   └── Images
├── horse
│   ├── Annotations
│   └── Images
└── laneline
    ├── Annotations
    └── Images

15 directories, 0 files


> 查看数据label的像素分布，可从中得知分割任务的类别数： 脚本位于: **show_segDataset_label_cls_id.py**

> 关于人像分割数据分析，这里不做提示，可参考[PaddleSeg实战——人像分割](https://aistudio.baidu.com/aistudio/projectdetail/2177440?channelType=0&channel=0)

In [ ]:
# 查看label中像素分类情况
!python show_segDataset_label_cls_id.py

100%|████████████████████████████████████████| 328/328 [00:00<00:00, 956.73it/s]
horse-cls_id:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255]
horse为90分类
horse实际应转换为2分类(将非0像素转换为像素值为1)


100%|████████████████████████████████████████| 845/845 [00:04<00:00, 190.33it/s]
facade-cls_id:  [0, 1, 2, 3, 4, 5, 6, 7, 8]
facade为9分类


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 179.01it/s]
fundusvessels-cls_id:  [0, 1]
fundusvessels为2分类


100%|███████████████████████████████████████| 4878/4878 [01:33<00:00, 52.37it/s]
laneline-cls_id:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
laneline为20分类


# 三、数据预处理

> 这里就以horse数据作为示例

-----

- 首先，通过上边的像素值分析以及horse本身的标签表现，我们确定horse数据集为二分类任务

- 然而，实际label中，却包含多个像素值，因此需要将horse中的所有label进行一个预处理

- 预处理内容为: 0值不变，非0值变为1，然后再保存label

- **并且保存文件格式为png，单通道图片为Label图片，最好保存为png——否则可能出现异常像素**

**对应horse的预处理脚本，位于:**

parse_horse_label.py

In [ ]:
!python parse_horse_label.py

100%|████████████████████████████████████████| 328/328 [00:00<00:00, 390.62it/s]
[0, 1]
100%|███████████████████████████████████████| 328/328 [00:00<00:00, 1110.42it/s]
horse-cls_id:  [0, 1]
horse为2分类


- 预处理完成后，配置训练的索引文件txt，方便后边套件读取数据

> txt创建脚本位于: **horse_create_train_list.py**

> 同时，生成的txt位于: **segDataset/horse/train_list.txt**

In [ ]:
# 创建训练的数据索引txt
# 格式如下
# line1: train_img1.jpg train_label1.png
# line2: train_img2.jpg train_label2.png
!python horse_create_train_list.py

100%|██████████████████████████████████████| 328/328 [00:00<00:00, 16339.24it/s]


# 四、使用套件开始训练

- 1.解压套件: 已挂载到本项目, 位于:**data/data102250/PaddleSeg-release-2.1.zip**

In [ ]:
# 解压套件
!unzip -oq data/data102250/PaddleSeg-release-2.1.zip
# 通过mv指令实现文件夹重命名
!mv PaddleSeg-release-2.1 PaddleSeg

- 2.选择模型，baseline选择**bisenet**, 位于: **PaddleSeg/configs/quick_start/bisenet_optic_disc_512x512_1k.yml**

- 3.配置模型文件

> 首先修改训练数据集加载的dataset类型:

![](https://ai-studio-static-online.cdn.bcebos.com/2f5363d71034490290f720ea8bb0d6873d7df2712d4b4e84ae41b0378aed8b89)

> 然后配置训练数据集如下:

![](https://ai-studio-static-online.cdn.bcebos.com/29547856db4b4bfc80aa3732e143f2788589f9316c694f369c9bd1da44b815dc)

> 类似的，配置验证数据集: -- **注意修改train_path为val_path**

![](https://ai-studio-static-online.cdn.bcebos.com/09713aaaed6b4611a525d25aae67e4f0538224f7ac0241eb941d97892bf6c4c1)

<font color="red" size=4>其它模型可能需要到: PaddleSeg/configs/$_base_$  中的数据集文件进行配置，但修改参数与bisenet中的数据集修改参数相同 </font>

![](https://ai-studio-static-online.cdn.bcebos.com/b154dcbf15e14f43aa13455c0ceeaaebe0489c9a09dd439f9d32e8b0a31355ec)


- 4.开始训练

使用PaddleSeg的train.py，传入模型文件即可开启训练

In [ ]:
!python PaddleSeg/train.py\
--config PaddleSeg/configs/quick_start/bisenet_optic_disc_512x512_1k.yml\
--batch_size 4\
--iters 2000\
--learning_rate 0.01\
--save_interval 200\
--save_dir PaddleSeg/output\
--seed 2021\
--log_iters 20\
--do_eval\
--use_vdl

# --batch_size 4\  # 批大小
# --iters 2000\    # 迭代次数 -- 根据数据大小，批大小估计迭代次数
# --learning_rate 0.01\ # 学习率
# --save_interval 200\ # 保存周期 -- 迭代次数计算周期
# --save_dir PaddleSeg/output\ # 输出路径
# --seed 2021\ # 训练中使用到的随机数种子
# --log_iters 20\ # 日志频率 -- 迭代次数计算周期
# --do_eval\ # 边训练边验证
# --use_vdl # 使用vdl可视化记录
# 用于断训==即中断后继续上一次状态进行训练
# --resume_model model_dir

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-08-15 11:07:23 [INFO]	
------------Environment Information-------------
platform: Linux-4.4.0-166-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: False
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddlePaddle: 2.0.2
OpenCV: 4.1.1
------------------------------------------------
2021-08-15 11:07:23 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 2000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  - 1
  types:
  - ignore_index: 255
    type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  pretrained: null
  type: BiSeNetV2
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_datas

In [ ]:
# 单独进行评估 -- 上边do_eval就是这个工作
!python PaddleSeg/val.py\
--config PaddleSeg/configs/quick_start/bisenet_optic_disc_512x512_1k.yml\
--model_path PaddleSeg/output/best_model/model.pdparams
# model_path： 模型参数路径

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-08-15 15:36:19 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 1000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  pretrained: null
  type: BiSeNetV2
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: segDataset/horse
  mode: train
  num_classes: 2
  train_path: segDataset/horse/train_list.txt
  transforms:
  - target_size:
    - 512
    - 512
    type: Resize
  - type: RandomHorizontalFlip
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: segDataset/horse
  mode: val
  num_classes: 2
  transforms:
  - target_size:
    - 512
    - 512
    type:

- 5.开始预测

In [ ]:
# 进行预测
!python PaddleSeg/predict.py\
--config PaddleSeg/configs/quick_start/bisenet_optic_disc_512x512_1k.yml\
--model_path PaddleSeg/output/best_model/model.pdparams\
--image_path segDataset/horse/Images\
--save_dir PaddleSeg/output/horse
# image_path: 预测图片路径/文件夹 -- 这里直接对训练数据进行预测，得到预测结果
# save_dir： 保存预测结果的路径 -- 保存的预测结果为图片

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2021-08-15 15:40:05 [INFO]	
---------------Config Information---------------
batch_size: 4
iters: 1000
loss:
  coef:
  - 1
  - 1
  - 1
  - 1
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  pretrained: null
  type: BiSeNetV2
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: segDataset/horse
  mode: train
  num_classes: 2
  train_path: segDataset/horse/train_list.txt
  transforms:
  - target_size:
    - 512
    - 512
    type: Resize
  - type: RandomHorizontalFlip
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: segDataset/horse
  mode: val
  num_classes: 2
  transforms:
  - target_size:
    - 512
    - 512
    type:

# 五、可视化预测结果

通过PaddleSeg预测输出的结果为图片，对应位于:PaddleSeg/output/horse

其中包含两种结果：

- 一种为掩膜图像，即叠加预测结果与原始结果的图像 -- 位于: **PaddleSeg/output/horse/added_prediction**
- 另一种为预测结果的伪彩色图像，即预测的结果图像 -- 位于: **PaddleSeg/output/horse/pseudo_color_prediction**

In [ ]:
# 查看预测结果文件夹分布
!tree PaddleSeg/output/horse -L 1

PaddleSeg/output/horse
├── added_prediction
└── pseudo_color_prediction

2 directories, 0 files



> 以下为展示结果

<font color='red' size=5> ---数据集 horse 的预测结果展示--- </font>

<font color='black' size=5> 掩膜图像： </font>

![](https://ai-studio-static-online.cdn.bcebos.com/31ce006a3a3f45378701a8f3064612c87555d827de3748fd9a98a1bc1efe2b4a)

<font color='black' size=5> 伪彩色图像： </font>

![](https://ai-studio-static-online.cdn.bcebos.com/8d6710c888954d1b92725e3defc9ae71cb07df5d271d40e6bf8581d6f7025ad6)


